<a href="https://colab.research.google.com/github/VitalyGladyshev/DevOps/blob/main/HW3_IAC_%D0%93%D0%BB%D0%B0%D0%B4%D1%8B%D1%88%D0%B5%D0%B2_%D0%92%D0%B8%D1%82%D0%B0%D0%BB%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Настройка окружения для работы с Ansible

In [1]:
%%capture
!sudo apt update
!sudo apt install -y ansible
!pip install ansible -qqq

### Настройка окружения для работы с OpenTofu



In [2]:
!curl --proto '=https' --tlsv1.2 -fsSL https://get.opentofu.org/install-opentofu.sh -o install-opentofu.sh
!chmod +x install-opentofu.sh
!./install-opentofu.sh --install-method deb
!rm -f install-opentofu.sh

OpenTofu Installer

Attempting installation via Debian repository...
Updating package list...
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubu

In [3]:
!which tofu

/usr/bin/tofu


Ansible поможет нам управлять инфраструктурой, а OpenTofu будет использоваться для описания и развертывания ресурсов.

In [10]:
%%writefile inventory.ini
[webservers]
localhost ansible_connection=local

Overwriting inventory.ini


In [5]:
%%writefile playbook.yaml
---
- name: Создаём инфраструктуру с OpenTofu
  hosts: local
  tasks:
    - name: Создать новый проект с OpenTofu
      command: tofu init

    - name: Посмотреть конфиг OpenTofu
      command: tofu show


Writing playbook.yaml


Выполним плейбук

In [6]:
!ansible-playbook -i inventory.ini playbook.yaml

/usr/lib/python3/dist-packages/winrm/__init__.py:105: SyntaxWarning: invalid escape sequence '\d'
  '(?i)^((?P<scheme>http[s]?)://)?(?P<host>[0-9a-z-_.]+)(:(?P<port>\d+))?(?P<path>(/)?(wsman)?)?', target)  # NOQA
/usr/lib/python3/dist-packages/requests_kerberos/kerberos_.py:85: SyntaxWarning: invalid escape sequence '\s'
  regex = re.compile('(?:.*,)*\s*Negotiate\s*([^,]*),?', re.I)
/usr/lib/python3/dist-packages/winrm/encryption.py:200: SyntaxWarning: invalid escape sequence '\w'
  if re.match('^.*-GCM-[\w\d]*$', cipher_suite):

PLAY [Создаём инфраструктуру с OpenTofu] ***************************************

TASK [Gathering Facts] *********************************************************
[WARNING]: Host 'localhost' is using the discovered Python interpreter at '/usr/bin/python3.12', but future installation of another Python interpreter could cause a different interpreter to be discovered. See https://docs.ansible.com/ansible-core/2.19/reference_appendices/interpreter_discovery.html f

In [19]:
!tofu show

No state.


## Задание 1. Отладка конфигурации Ansible

Задача: в конфигурации Ansible нет файла инвентаря. Нужно прописать файл инвентаря, чтобы плейбук выполнялся корректно.


In [11]:
%%writefile playbook.yaml
---
- name: Установка и запуск веб-сервера
  hosts: webservers
  become: yes
  tasks:
    - name: Установка Apache
      apt:
        name: apache2
        state: present
        update_cache: yes
    - name: Запуск и включение Apache
      service:
        name: apache2
        state: started
        enabled: yes

Overwriting playbook.yaml


### Исправление ошибок в плейбуке

1. Убедитесь, что Ansible установлен и доступен для выполнения команд.
2. Проверьте, что файл инвентаря и плейбук созданы корректно.
3. Запустите плейбук и исправьте ошибки, если они возникнут.

При корректном запуске в итоговой строке PLAY RECAP будет указано количество выполненных без ошибок этапов ok=3

In [12]:
!ansible-playbook -i inventory.ini playbook.yaml # Выполняем плейбук для установки и запуска Apache


PLAY [Установка и запуск веб-сервера] ******************************************

TASK [Gathering Facts] *********************************************************
[WARNING]: Host 'localhost' is using the discovered Python interpreter at '/usr/bin/python3.12', but future installation of another Python interpreter could cause a different interpreter to be discovered. See https://docs.ansible.com/ansible-core/2.19/reference_appendices/interpreter_discovery.html for more information.
ok: [localhost]

TASK [Установка Apache] ********************************************************
changed: [localhost]

TASK [Запуск и включение Apache] ***********************************************
changed: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=3    changed=2    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



## Задание 2. Отладка конфигурации OpenTofu

Задача: конфигурация OpenTofu содержит опечатку. Исправьте ошибку, чтобы конфигурация проходила валидацию.

In [13]:
%%writefile main.tf
terraform {
  required_providers {
    null = {
      source = "hashicorp/null"
      version = ">= 3.0"
    }
  }
}

data "null_data_source" "example" {
  inputs = {
    location = "ru-1"
    disk = 18 * 1024
    type = "postgres"                # <- как нужно правильно оформить строковый лиерал ?
  }
}

resource "null_resource" "example" {
  provisioner "local-exec" {
    command = "echo 'Создан требуемый ресурс'"
  }
}

Writing main.tf


In [14]:
!tofu init


Initializing the backend...

Initializing provider plugins...
- Finding hashicorp/null versions matching ">= 3.0.0"...
- Installing hashicorp/null v3.2.4...
- Installed hashicorp/null v3.2.4 (signed, key ID 0C0AF313E5FD9F80)

Providers are signed by their developers.
If you'd like to know more about provider signing, you can read about it here:
https://opentofu.org/docs/cli/plugins/signing/

OpenTofu has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that OpenTofu can guarantee to make the same selections by default when
you run "tofu init" in the future.

OpenTofu has been successfully initialized!

You may now begin working with OpenTofu. Try running "tofu plan" to see
any changes that are required for your infrastructure. All OpenTofu commands
should now work.

If you ever set or change modules or backend configuration for OpenTofu,
rerun this command to reinitialize your working direc

### Исправление ошибок в конфигурации OpenTofu

1. Убедитесь, что OpenTofu установлен и доступен для выполнения команд.
2. Проверьте корректность конфигурационного файла.

Валидация успешна, если есть сообщение `Success!` The configuration is valid, but there were some validation warnings as shown above.



## Использование логов и отладочных инструментов

Логи и отладочные инструменты помогают глубже понять, что происходит в процессе выполнения команд Ansible и OpenTofu. Используйте их для выявления и устранения проблем.

In [15]:
# Включение детализированных логов для Ansible
!ansible-playbook -i inventory.ini playbook.yaml -v # Запускаем плейбук с детализированными логами

No config file found; using defaults

PLAY [Установка и запуск веб-сервера] ******************************************

TASK [Gathering Facts] *********************************************************
[WARNING]: Host 'localhost' is using the discovered Python interpreter at '/usr/bin/python3.12', but future installation of another Python interpreter could cause a different interpreter to be discovered. See https://docs.ansible.com/ansible-core/2.19/reference_appendices/interpreter_discovery.html for more information.
ok: [localhost]

TASK [Установка Apache] ********************************************************
ok: [localhost] => {"cache_update_time": 1762624273, "cache_updated": false, "changed": false}

TASK [Запуск и включение Apache] ***********************************************
ok: [localhost] => {"changed": false, "enabled": true, "name": "apache2", "state": "started"}

PLAY RECAP *********************************************************************
localhost                 

In [16]:
!export TOFU_LOG=DEBUG # Устанавливаем уровень логирования на DEBUG для OpenTofu
!tofu validate

╷
│ Warning: Deprecated
│ 
│   with data.null_data_source.example,
│   on main.tf line 10, in data "null_data_source" "example":
│   10: data "null_data_source" "example" {
│ 
│ The null_data_source was historically used to construct intermediate values to re-use elsewhere in configuration, the same can now be achieved using locals or the terraform_data resource type in Terraform 1.4 and later.
╵
Success! The configuration is valid, but there were some validation warnings as shown above.


## Задание 3. Работа с оберткой Tofupy

Задачи:
1. Подключить библиотеку-обертку Tofupy через import.
2. Указать рабочую директорию, путь до бинарного файла.
3. Вызвать метод workspace.validate для проверки конфигурации на наличие ошибок.

- Если в конфигурации есть ошибка, то будет сообщение «Конфигурация некорректна».
- Если нет ошибок, то будет сообщение «Конфигурация некорректна».


In [20]:
!pip install tofupy -qq
!mkdir tofudir

In [21]:

import os
from tofupy import Tofu

workspace = Tofu(cwd="./tofudir",     # Указываем рабочую директорию
    binary="tofu",                    # Указываем бинарный файл OpenTofu
    log_level="DEBUG",
    env={"TF_VAR_environment": "production"} #
)
workspace.init()

# ВАШ КОД ЗДЕСЬ
validation =  workspace.validate()                    # <-вызовите метод проверки конфигурации на наличие ошибок
# ВАШ КОД ЗДЕСЬ

if not validation.valid:
    print("конфигурация некорректна!")
    for diagnostic in validation.diagnostics:
        print(f"ошибки конфигурации: {diagnostic.summary}")
else:
    print("конфигурация корректна!")



конфигурация корректна!


## Задание 4. Программное создание бакета S3

Задача: на языке HCL (Terraform/Tofu) описать холодное хранилище на 20 Гб, используя S3.

Чтобы на языке HCL (Terraform/Tofu) описать холодное хранилище на 20 Гб, используя S3, нужно указать disk = 20 * 1024

Формат описания приведен в конспекте занятия.


In [30]:
%%writefile main.tf
terraform {
  required_providers {
    null = {
      source = "hashicorp/null"
      version = ">= 3.0"
    }
  }
}

data "null_data_source" "example" {
  inputs = {
    location = "us-east-1"
    storage_class = "GLACIER"
    disk_size_gb = 20
    disk_size_mb = 20 * 1024
    type = "s3-cold-storage"
    bucket_name = "my-cold-storage-bucket-12345"
  }
}

resource "null_resource" "example" {
  provisioner "local-exec" {
    command = "echo 'Создано холодное S3 хранилище: регион ${data.null_data_source.example.outputs["location"]}, класс хранения ${data.null_data_source.example.outputs["storage_class"]}, размер ${data.null_data_source.example.outputs["disk_size_gb"]} ГБ'"
  }
}

Overwriting main.tf


In [25]:
!tofu init


Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/null from the dependency lock file
- Using previously-installed hashicorp/null v3.2.4

OpenTofu has been successfully initialized!

You may now begin working with OpenTofu. Try running "tofu plan" to see
any changes that are required for your infrastructure. All OpenTofu commands
should now work.

If you ever set or change modules or backend configuration for OpenTofu,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [26]:
!tofu apply

data.null_data_source.example: Reading...
data.null_data_source.example: Read complete after 0s [id=static]

OpenTofu used the selected providers to generate the following execution plan. Resource actions are indicated with the following symbols:
  + create

OpenTofu will perform the following actions:

  # null_resource.example will be created
  + resource "null_resource" "example" {
      + id = (known after apply)
    }

Plan: 1 to add, 0 to change, 0 to destroy.
╷
│ Warning: Deprecated
│ 
│   with data.null_data_source.example,
│   on main.tf line 10, in data "null_data_source" "example":
│   10: data "null_data_source" "example" {
│ 
│ The null_data_source was historically used to construct intermediate values to re-use elsewhere in configuration, the same can now be achieved using locals or the terraform_data resource type in Terraform 1.4 and later.
│ 
│ (and one more similar warning elsewhere)
╵

Do you want to perform these actions?
  OpenTofu will perform the actions describe

## Задание 5. Программное создание базы данных

Задача: создайте пресет базы PostgreSQL с диском на 80 ГБ и экземпляр  базы PostgreSQL с логином/паролем user_010101/888-fff-888 на языке HCL.

Чтобы на языке HCL (Terraform/Tofu) описать базу PostgreSQL с диском на 80 ГБ,  нужно указать:
* disk = 80 * 1024
* login = " user_010101"
* password = "888-fff-888”.

Формат описания приведен в конспекте занятия.


In [27]:
%%writefile main.tf
terraform {
  required_providers {
    null = {
      source = "hashicorp/null"
      version = ">= 3.0"
    }
  }
}

data "null_data_source" "postgres_preset" {
  inputs = {
    disk_size_gb = 80
    disk_size_mb = 80 * 1024
    engine = "PostgreSQL"
    engine_version = "13"
    instance_class = "db.t3.micro"
  }
}

data "null_data_source" "postgres_instance" {
  inputs = {
    login = "user_010101"
    password = "888-fff-888"
    database_name = "app_database"
    port = 5432
  }
}

resource "null_resource" "postgres_setup" {
  provisioner "local-exec" {
    command = "echo 'Создан пресет PostgreSQL: диск ${data.null_data_source.postgres_preset.outputs["disk_size_gb"]} ГБ, экземпляр с логином ${data.null_data_source.postgres_instance.outputs["login"]} и паролем ${data.null_data_source.postgres_instance.outputs["password"]}'"
  }
}


Overwriting main.tf


In [28]:
!tofu init


Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/null from the dependency lock file
- Using previously-installed hashicorp/null v3.2.4

OpenTofu has been successfully initialized!

You may now begin working with OpenTofu. Try running "tofu plan" to see
any changes that are required for your infrastructure. All OpenTofu commands
should now work.

If you ever set or change modules or backend configuration for OpenTofu,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [29]:
!tofu apply

data.null_data_source.postgres_preset: Reading...
null_resource.example: Refreshing state... [id=5554812604176471202]
data.null_data_source.postgres_instance: Reading...
data.null_data_source.postgres_preset: Read complete after 0s [id=static]
data.null_data_source.postgres_instance: Read complete after 0s [id=static]

OpenTofu used the selected providers to generate the following execution plan. Resource actions are indicated with the following symbols:
  + create
  - destroy

OpenTofu will perform the following actions:

  # null_resource.example will be destroyed
  # (because null_resource.example is not in configuration)
  - resource "null_resource" "example" {
      - id = "5554812604176471202" -> null
    }

  # null_resource.postgres_setup will be created
  + resource "null_resource" "postgres_setup" {
      + id = (known after apply)
    }

Plan: 1 to add, 0 to change, 1 to destroy.
╷
│ Warning: Deprecated
│ 
│   with data.null_data_source.postgres_preset,
│   on main.tf line 1

### Задание 6. Итоговое оформление

Подготовьте ноутбук в логичной структуре:
- написание кода;
- запуск кода;
- текстовые пояснения к шагам (в markdown-ячейках, 5–8 предложений на каждый раздел);
- общий вывод.

В итоговых выводах дайте 5–8 предложений о своем опыте работы с инструментами модуля: что оказалось простым, что вызвало трудности, какие выводы сделали о применимости подхода IaC в реальных проектах.



В ходе выполнения задач была успешно попробовал работать с инструментами Infrastructure as Code (IaC) - OpenTofu и Ansible. Настроил автоматизированную среду развертывания и создал конфигурации для управления инфраструктурой. Подход с использованием null-провайдера позволил протестировать логику конфигураций без создания реальных облачных ресурсов.

Infrastructure as Code демонстрирует эффективность в реальных проектах, обеспечивая воспроизводимость и согласованность инфраструктуры. Данный подход значительно снижает риски человеческих ошибок при ручном развертывании и упрощает процесс аудита изменений. IaC особенно ценен в DevOps-практиках, позволяя автоматизировать полный цикл развертывания от разработки до продакшена.